In [4]:
import dask
from fake_useragent import UserAgent
from SPARQLWrapper import SPARQLWrapper, JSON, XML
import pandas as pd
import json
import time
import bookquery
import os
import io
import json
from tqdm import tqdm
from os.path import join,isfile
from os import listdir
from collections import Counter
import dask
import openpyxl

def counter_for_files(path):
    
    """Подсчет файлов в директории"""
        
    onlyfiles = [join(path, f) for f in listdir(path) if isfile(join(path, f))]
    return onlyfiles

In [24]:
ua = UserAgent()
bad_requests = []

def execute_query(qquery, entity_id):
    
    """Базовая функция для связи с сервером"""
    
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql", agent = ua.random)
    sparql.setQuery(qquery.format(entity_id = entity_id))
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return results['results']['bindings']

def execute_query_2_args(qquery, entity_id, country_id):
    
    """Базовая функция для связи с сервером"""
    
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql", agent = ua.random)
    sparql.setQuery(qquery.format(entity_id = entity_id,country_id = country_id))
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return results['results']['bindings']

def entities_sparqul(entity_id, f = 0):
    
    """Получение ответа с сервера и формирование датафреймов"""
    
    res = []
    if f > 5:
        global bad_requests
        bad_requests.append(entity_id)
        print('Bad request ', entity_id)
        return None
    try:
        results= execute_query(bookquery.leaders, entity_id)
        for result in results:
            entities = {}
            entities['leader'] = result['leader']['value'].split('/')[-1]
            res.append(entities)
        return pd.DataFrame(data = res, columns = ['leader'])
    except Exception as e:
        print(e)
        time.sleep(5)
        print('Try again ', entity_id, f)
        return entities_sparqul(entity_id, f + 1)
    
def positions(entity_id, country_id, f = 0):
    """Получение ответа с сервера и формирование датафреймов"""
    
    res = []
    if f > 5:
        global bad_requests
        bad_requests.append(entity_id)
        print('Bad request ', entity_id)
        return None
    try:
        results= execute_query_2_args(bookquery.leaders_position, entity_id,country_id)
        for result in results:
            entities = {}
            entities['person'] = result['person']['value'].split('/')[-1]
            entities['personLabel'] = result['personLabel']['value']
            entities['position'] = result['position']['value'].split('/')[-1]
            entities['positionLabel'] = result['positionLabel']['value']
            try:
                entities['startTime'] = result['startTime']['value']
                entities['startTimeNode'] = result['startTimeNode']['value']
            except:
                entities['startTime'] = "не указано"
                entities['startTimeNode'] = "не указано"
            try:
                entities['endTime'] = result['endTime']['value']
                entities['endTimeNode'] = result['endTimeNode']['value']
            except:
                entities['endTime'] = "не указано"
                entities['endTimeNode'] = "не указано"
            res.append(entities)
        return pd.DataFrame(data = res, columns = ['person', 'personLabel' ,'position' ,'positionLabel' ,'startTime' ,'startTimeNode','endTime' , 'endTimeNode'])
    except Exception as e:
        print(e)
        time.sleep(5)
        print('Try again ', entity_id, f)
        return entities_sparqul(entity_id, f + 1)
    
def file_creation(dic,rulers):
    for ID,name in dic.items():
        new_df = pd.DataFrame()
        data = entities_sparqul(ID)
        for k,v in data.iterrows():
            new_df = pd.concat([new_df,positions(v.leader,ID)])
        new_df['real_ruler'] = 'F'
        new_df['Delete'] = 'F'
        new_df.loc[new_df.position == rulers[ID],"real_ruler"] = "T"
        new_df.to_excel(f'Rulers/{name}.xlsx', sheet_name = name ,encoding='utf-8-sig' , index = False)
    return new_df

In [25]:
dic_id = {"Q15180" : 'СССР' ,"Q159":"Россия", "Q183":"Германия" , "Q30":"США" , "Q230":"Грузия"}
dic_real_rulers = {"Q15180" : 'Q1048744' ,"Q159":"Q218295", "Q183":"Q4970706" , "Q30":"Q11696" , "Q230":"Q843405"}
df = file_creation(dic_id , dic_real_rulers)

In [18]:
df.loc[df.position == dic_real_rulers["Q230"],"real_ruler"] = "T"

In [19]:
df

,person,personLabel,position,positionLabel,startTime,startTimeNode,endTime,endTimeNode,real_ruler,Delete
0,Q6066533,"Гарибашвили, Ираклий",Q1975365,Список премьер-министров Грузии,2013-11-20T00:00:00Z,http://www.wikidata.org/value/23a89ece5f28f074...,2015-12-30T00:00:00Z,http://www.wikidata.org/value/27ab69969fbf3aea...,F,F
0,Q15277881,Георгий Квирикашвили,Q1975365,Список премьер-министров Грузии,2015-12-30T00:00:00Z,http://www.wikidata.org/value/27ab69969fbf3aea...,2018-06-13T00:00:00Z,http://www.wikidata.org/value/7b3bc14b466ea65a...,F,F
1,Q15277881,Георгий Квирикашвили,Q21290878,депутат Парламента Грузии,не указано,не указано,не указано,не указано,F,F
0,Q54982996,Мамука Бахтадзе,Q1975365,Список премьер-министров Грузии,2018-06-20T00:00:00Z,http://www.wikidata.org/value/34dd62720a017aeb...,не указано,не указано,F,F
0,Q57794,"Иванишвили, Бидзина Григорьевич",Q1975365,Список премьер-министров Грузии,2012-10-25T00:00:00Z,http://www.wikidata.org/value/cf2baaecb4af7a38...,2013-11-20T00:00:00Z,http://www.wikidata.org/value/23a89ece5f28f074...,F,F
0,Q151929,Эдуард Амвросиевич Шеварднадзе,Q843405,президент Грузии,1995-11-26T00:00:00Z,http://www.wikidata.org/value/c92bfa071ff48ad8...,2003-11-23T00:00:00Z,http://www.wikidata.org/value/6765fa50527b5989...,T,F
0,Q218435,Звиад Константинович Гамсахурдия,Q3652076,председатель Парламента Грузии,1990-11-14T00:00:00Z,http://www.wikidata.org/value/4fe676d14d48b0b6...,1991-04-14T00:00:00Z,http://www.wikidata.org/value/8339718a371a4ebc...,F,F
1,Q218435,Звиад Константинович Гамсахурдия,Q843405,президент Грузии,1991-04-14T00:00:00Z,http://www.wikidata.org/value/8339718a371a4ebc...,1992-01-06T00:00:00Z,http://www.wikidata.org/value/e386926d740d9369...,T,F
0,Q229493,Нино Анзоровна Бурджанадзе,Q21290878,депутат Парламента Грузии,не указано,не указано,не указано,не указано,F,F
1,Q229493,Нино Анзоровна Бурджанадзе,Q843405,президент Грузии,2003-11-23T00:00:00Z,http://www.wikidata.org/value/6765fa50527b5989...,2004-01-25T00:00:00Z,http://www.wikidata.org/value/85a1421535721582...,T,F
